<a href="https://colab.research.google.com/github/jsaulme/apps/blob/main/pso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
code = """
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# Configuración de Streamlit
st.title("Visualización de PSO para el Problema del Agente Viajero")

# Controles de usuario
# Número de ciudades: elige la cantidad de ciudades que deberá recorrer el viajero
num_ciudades = st.slider("Número de Ciudades que Recorrerá el Viajero", 5, 50, 10)
# Número de partículas: el número de soluciones candidatas en cada iteración
num_particulas = st.slider("Número de Partículas (soluciones candidatas)", 10, 100, 30)
# Número de iteraciones: el número de veces que el algoritmo actualizará las soluciones
iteraciones = st.slider("Número de Iteraciones", 1, 200, 50)
# Coeficiente de inercia: controla la influencia de la velocidad anterior en la nueva velocidad
coef_inercia = st.slider("Coeficiente de Inercia (influencia de la velocidad anterior en la nueva velocidad)", 0.0, 1.0, 0.5)
# Coeficiente cognitivo: controla la influencia de la mejor solución personal en la nueva velocidad
coef_cognitivo = st.slider("Coeficiente Cognitivo (influencia de la mejor solución personal en la nueva velocidad)", 0.0, 2.0, 1.0)
# Coeficiente social: controla la influencia de la mejor solución global en la nueva velocidad
coef_social = st.slider("Coeficiente Social (influencia de la mejor solución global en la nueva velocidad)", 0.0, 2.0, 1.0)

# Definir el problema del agente viajero
def distancia(ciudad1, ciudad2):
    return np.linalg.norm(np.array(ciudad1) - np.array(ciudad2))

def costo_ruta(ruta, ciudades):
    return sum(distancia(ciudades[ruta[i]], ciudades[ruta[i + 1]]) for i in range(len(ruta) - 1))

# Inicialización
ciudades = np.random.rand(num_ciudades, 2)
particulas = [np.random.permutation(num_ciudades) for _ in range(num_particulas)]
velocidades = [np.zeros(num_ciudades) for _ in range(num_particulas)]
mejor_personal = particulas.copy()
mejor_global = min(particulas, key=lambda p: costo_ruta(p, ciudades))

# PSO
historial_costos = []

for _ in range(iteraciones):
    for i in range(num_particulas):
        if costo_ruta(particulas[i], ciudades) < costo_ruta(mejor_personal[i], ciudades):
            mejor_personal[i] = particulas[i].copy()
        if costo_ruta(mejor_personal[i], ciudades) < costo_ruta(mejor_global, ciudades):
            mejor_global = mejor_personal[i].copy()
        # Actualizar velocidad y posición
        velocidades[i] = (coef_inercia * velocidades[i] +
                          coef_cognitivo * np.random.rand() * (mejor_personal[i] - particulas[i]) +
                          coef_social * np.random.rand() * (mejor_global - particulas[i]))
        particulas[i] = np.argsort(particulas[i] + velocidades[i])
    historial_costos.append(costo_ruta(mejor_global, ciudades))

# Visualización de las ciudades y la mejor ruta encontrada
fig, ax = plt.subplots()
# Graficar las ciudades como puntos rojos
ax.scatter(ciudades[:, 0], ciudades[:, 1], c='red')
# Anotar cada ciudad con su índice
for i in range(num_ciudades):
    ax.annotate(f'{i}', (ciudades[i, 0], ciudades[i, 1]))
# Crear la ruta cerrada añadiendo la primera ciudad al final
ruta = np.append(mejor_global, mejor_global[0])
# Graficar la ruta como una línea azul
ax.plot(ciudades[ruta, 0], ciudades[ruta, 1], 'b-')
# Mostrar el gráfico en Streamlit
st.pyplot(fig)

# Gráfico de convergencia del costo
fig2, ax2 = plt.subplots()
# Graficar el historial de costos en cada iteración
ax2.plot(historial_costos, 'g-')
ax2.set_title("Convergencia del Costo")
ax2.set_xlabel("Iteraciones")
ax2.set_ylabel("Costo")
# Mostrar el gráfico de convergencia en Streamlit
st.pyplot(fig2)

# Resultados
st.write("Mejor ruta encontrada:", mejor_global)
st.write("Costo de la mejor ruta:", costo_ruta(mejor_global, ciudades))
"""

with open("app.py", "w") as file:
    file.write(code)


In [3]:
# Listar archivos en el directorio actual
!ls -l

# Visualizar el contenido del archivo app.py
!cat app.py


total 8
-rw-r--r-- 1 root root 3766 Aug  5 23:35 app.py
drwxr-xr-x 1 root root 4096 Aug  2 13:24 sample_data

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# Configuración de Streamlit
st.title("Visualización de PSO para el Problema del Agente Viajero")

# Controles de usuario
# Número de ciudades: elige la cantidad de ciudades que deberá recorrer el viajero
num_ciudades = st.slider("Número de Ciudades que Recorrerá el Viajero", 5, 50, 10)
# Número de partículas: el número de soluciones candidatas en cada iteración
num_particulas = st.slider("Número de Partículas (soluciones candidatas)", 10, 100, 30)
# Número de iteraciones: el número de veces que el algoritmo actualizará las soluciones
iteraciones = st.slider("Número de Iteraciones", 1, 200, 50)
# Coeficiente de inercia: controla la influencia de la velocidad anterior en la nueva velocidad
coef_inercia = st.slider("Coeficiente de Inercia (influencia de la velocidad anterior en la nueva velocidad)", 0.0, 1

In [4]:
import subprocess
from pyngrok import ngrok

# Reemplaza 'TU_AUTHTOKEN' con tu authtoken real
ngrok.set_auth_token("2jgL8hf4dtW4lkLgnOQAZDxpYWI_67nG5dqrgL15AkymbriZ")

# Get your authtoken from https://dashboard.ngrok.com/auth
!ngrok authtoken <2jgL8hf4dtW4lkLgnOQAZDxpYWI_67nG5dqrgL15AkymbriZ>

# Inicia el servidor Streamlit
process = subprocess.Popen(['streamlit', 'run', 'app.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Crea un túnel para acceder a la aplicación, especificando el puerto en el parámetro addr
public_url = ngrok.connect(addr="8501") # Change here: Use addr="8501" instead of port=8501
print(f'Public URL: {public_url}')

# Espera un momento para que Streamlit se inicie
import time
time.sleep(10)

# Mostrar logs de Streamlit (esto puede necesitar ajustes dependiendo de cómo quieres acceder a los logs)
# print(process.stdout.read().decode('utf-8'))

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `ngrok authtoken <2jgL8hf4dtW4lkLgnOQAZDxpYWI_67nG5dqrgL15AkymbriZ>'
Public URL: NgrokTunnel: "https://03f0-34-106-220-152.ngrok-free.app" -> "http://localhost:8501"
